In [16]:
import os
import torch
import trimesh
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [17]:
# Define the IM-NET Decoder
class IMNetDecoder(nn.Module):
    def __init__(self, latent_dim=128, hidden_dim=512):
        super(IMNetDecoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim + 3, hidden_dim)  # Input layer: latent vector + 3D points
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)      # Hidden layer
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)      # Another hidden layer
        self.fc4 = nn.Linear(hidden_dim, 1)              # Output layer: scalar output (0 or 1)
    
    def forward(self, p, z):
        x = torch.cat([p, z], dim=-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

In [18]:
# Define the Encoder (3D Convolutional Network)
class Encoder3D(nn.Module):
    def __init__(self, input_dim=64, latent_dim=128):
        super(Encoder3D, self).__init__()
        self.conv1 = nn.Conv3d(1, 32, 4, stride=2, padding=1)
        self.conv2 = nn.Conv3d(32, 64, 4, stride=2, padding=1)
        self.conv3 = nn.Conv3d(64, 128, 4, stride=2, padding=1)
        self.conv4 = nn.Conv3d(128, 256, 4, stride=2, padding=1)
        self.fc = nn.Linear(256 * 4 * 4 * 4, latent_dim)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)  # Latent vector
        return x

In [19]:
# ShapeDataset Class to Load Data from OFF Files
class ShapeDataset(Dataset):
    def __init__(self, data_dir, points_per_shape=10000, resolution=32):
        self.data_dir = data_dir
        self.points_per_shape = points_per_shape
        self.resolution = resolution
        self.model_files = [os.path.join(self.data_dir, f)
                            for f in os.listdir(data_dir) if f.endswith('.off')]
    
    def __len__(self):
        return len(self.model_files)
    
    def __getitem__(self, idx):
        model_file = self.model_files[idx]
        mesh = trimesh.load(model_file)

        voxel_data = mesh.voxelized(pitch=1.0 / self.resolution)
        voxel_grid = voxel_data.dense

        points = np.random.uniform(-1, 1, size=(self.points_per_shape, 3))
        inside_outside = np.zeros(self.points_per_shape)

        for i, point in enumerate(points):
            voxel_coords = ((point + 1) * 0.5 * (self.resolution - 1)).astype(int)
            voxel_coords = np.clip(voxel_coords, 0, self.resolution - 1)
            inside_outside[i] = voxel_grid[voxel_coords[0], voxel_coords[1], voxel_coords[2]]

        points = torch.FloatTensor(points)
        inside_outside = torch.FloatTensor(inside_outside).unsqueeze(1)

        return points, inside_outside

In [20]:
# Paths to training and testing data directories
train_data_dir = r'D:\Code\Python\ANK\IN NET\sano data\train'
test_data_dir = r'D:\Code\Python\ANK\IN NET\sano data\test'

In [21]:
# Create datasets for training and testing
train_dataset = ShapeDataset(data_dir=train_data_dir, points_per_shape=1000, resolution=32)
test_dataset = ShapeDataset(data_dir=test_data_dir, points_per_shape=1000, resolution=32)

In [22]:
# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [23]:
# Training Loop
def train(model, encoder, dataloader, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for points, inside_outside in dataloader:
            optimizer.zero_grad()

            z = encoder(points)  # Get latent vector from the encoder
            pred = model(points, z)  # Get predictions from the decoder

            loss = F.mse_loss(pred, inside_outside)  # Calculate loss
            loss.backward()  # Backpropagate the loss
            optimizer.step()  # Update model parameters

        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

In [24]:
# Instantiate and train the model
latent_dim = 128
decoder = IMNetDecoder(latent_dim=latent_dim)
encoder = Encoder3D(latent_dim=latent_dim)
optimizer = torch.optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=0.001)

In [27]:

# Train the model
train(decoder, encoder, train_dataloader, optimizer, num_epochs=5)

MemoryError: Unable to allocate 1.58 GiB for an array with shape (2, 105787392) and data type uint64